## Exceptions

What happens when your program attempts to do something that just can't be done?

This should not be normal.  It should be rare!  Typically happens when your program encounters and *exceptional* situation

In [1]:
3/0

ZeroDivisionError: division by zero

In [2]:
str.lower(1)

TypeError: descriptor 'lower' requires a 'str' object but received a 'int'

In [3]:
""[3]

IndexError: string index out of range

In [4]:
3 % 0

ZeroDivisionError: integer division or modulo by zero

Q: What should a function do?

A: One thing well.

Q: What should it do if it is passed arguments that don't make sense?

In [5]:
def divides(x, y):
    return y%x == 0

In [6]:
def get(data, selector):
    return data[selector]

In [7]:
divides(2,4)

True

In [8]:
divides(0,5)

ZeroDivisionError: integer division or modulo by zero

In [9]:
get([1,2,3],0)

1

In [10]:
get({'a': 34, 'cat':'9 lives'}, 'dog')

KeyError: 'dog'

In [11]:
get([1,2,3],[2])

TypeError: list indices must be integers or slices, not list

When an error is encountered the python interpreter *throws an exception*.  Here returns all the way to the top level and reports a stack trace of where the exception occured.

In [12]:
def divides(x, y):
    return y%x == 0
def divides24(x):
    return divides(x,24)
divides24(0)

ZeroDivisionError: integer division or modulo by zero

In [13]:
divides24(0)

ZeroDivisionError: integer division or modulo by zero

In [14]:
def mapply(f, s):
    return [f(x) for x in s]

In [15]:
mapply(divides24,[6,4,3,5])

[True, True, True, False]

In [16]:
mapply(divides24,[6,0,4,3,5])

ZeroDivisionError: integer division or modulo by zero

Many types of exceptions:

* `TypeError` -- A function was passed the wrong number/type of argument
* `NameError` -- A name wasn't found
* `KeyError` -- A key wasn't found in a dictionary
* `RuntimeError` -- Catch-all for troubles during interpretation

The flow of control stops at the exception and is 'thrown back'. Here the return (and the print) is not executed if an exception occurs on the divide.

In [17]:
def noisy_divides(x, y):
    result = (y % x == 0)
    if result:
        print("{0} divides {1}".format(x, y))
    else:
        print("{0} does not divide {1}".format(x, y))
    return result

In [18]:
noisy_divides(4,24)

4 divides 24


True

In [19]:
noisy_divides(0,24)

ZeroDivisionError: integer division or modulo by zero

In [20]:
def divides24(x):
    return noisy_divides(x,24)

In [21]:
divides24(0)

ZeroDivisionError: integer division or modulo by zero

## Assertions

Your functions should do all they can to avoid errors, they should handle them gracefully when they occur, and the should not trust that they are called with valid arguments -
*treat data as dirty till you've washed it*.

The most common form of throwing exceptions is with the `assert` statement.  Use it generously. Make sure that you code is working on something reasonable before it tries to do its stuff.  It serves as good documentation of the assumptions that your code makes.  And it avoids lots of very obscure bugs.

    asset <assertion expression>, <string for failed assertion>
    
Assert statements raise an exception of type `AssertionError`

In [22]:
def divides(x, y):
    assert x != 0, "Bad argument to divides - denominator should be non-zero"
    assert (type(x) == int and type(y) == int), "divides only takes integers"
    return y%x == 0

In [23]:
divides(0,3)

AssertionError: Bad argument to divides - denominator should be non-zero

In [24]:
divides(9, "lives")

AssertionError: divides only takes integers

In [25]:
def divides24(x):
    return divides(x,24)

In [26]:
mapply(divides24,[6,0,4,3,5])

AssertionError: Bad argument to divides - denominator should be non-zero

In [27]:
__debug__

True

## Handling errors

How can you continue in the presence of an error?  Is there a way to *handle the exception*?

The general form of this construct is

    try:
        <try suite>
    except <exception class> as <name>:
        <except suite>
    ... # continue here if <try suite> succeeds without exception

In [28]:
def safe_apply_fun(f,x):
    try:
        return f(x)   # normal execution, return the result
    except:           # error occured, f cannot return.  Transfer control back to here
        return "Invalid"   # value returned on exception

In [29]:
def divides(x, y):
    return y%x == 0
def divides24(x):
    return divides(x,24)
safe_apply_fun(divides24,0)

'Invalid'

In [30]:
def mapply(f, s):
    return [safe_apply_fun(f,x) for x in s]

In [31]:
mapply(divides24,[6,0,4,3,5])

[True, 'Invalid', True, True, False]

In [32]:
def rapply(f, s):
    if len(s) == 0:
        return []
    else:
        return [f(s[0])] + rapply(f, s[1:])

In [33]:
rapply(divides24, [6,4,3,5])

[True, True, True, False]

In [34]:
rapply(divides24, [6,4,3,0,5])

ZeroDivisionError: integer division or modulo by zero

In [35]:
def rapply(f, s):
    if len(s) == 0:
        return []
    else:
        return [safe_apply_fun(f, s[0])] + rapply(f, s[1:])

In [36]:
rapply(divides24, [6,4,3,0,5])

[True, True, True, 'Invalid', False]

In [37]:
def safe_apply_fun(f,x):
    try:
        return f(x)   # normal execution, return the result
    except Exception as e:  # exceptions are objects of class derived from base class Exception
        return e   # value returned on exception

In [38]:
safe_apply_fun(divides24,0)

ZeroDivisionError('integer division or modulo by zero')

In [39]:
res = mapply(divides24, [6,4,3,0,5])
res

[True,
 True,
 True,
 ZeroDivisionError('integer division or modulo by zero'),
 False]

In [40]:
res[3]

ZeroDivisionError('integer division or modulo by zero')

In [41]:
type(res[3])

ZeroDivisionError

## More on except

Execution rule:
The <try suite> is executed first
If, during the course of executing the <try suite>,
an exception is raised that is not handled otherwise, and
If the class of the exception inherits from <exception class>, then
The <except suite> is executed, with <name> bound to the exception

* There can be more than one `except` clause for a `try`.
* They may specify a tuple of exception types.
* The first one that catches the exception receives control.
* If none do (or if there is no `try ... except`) control is thrown out of the function call.
* Each of the function calls on the stack may define exception handlers.  Control is transferred to nearest catching exception suite on the stack.

In [42]:
def safe_apply_fun(f,x):
    try:
        return f(x)   # normal execution, return the result
    except AssertionError as e:
        return "Failed Assertion"
    except (TypeError, NameError):
        return "Bad function or arg type"

In [43]:
safe_apply_fun(divides24, 0)

ZeroDivisionError: integer division or modulo by zero

In [44]:
safe_apply_fun("foo", 0)

'Bad function or arg type'

In [45]:
safe_apply_fun(divides25, 0)

NameError: name 'divides25' is not defined

In [46]:
safe_apply_fun(lambda x: 24 % x == 0, 0)

ZeroDivisionError: integer division or modulo by zero

## Raising your own exceptions

Exceptions are raised with a raise statement
raise <expression>
<expression> must evaluate to a subclass of BaseException or an instance of one
Exceptions are constructed like any other object. E.g., `TypeError('Bad argument!')`

In [47]:
TypeError("ugly type")

TypeError('ugly type')

In [48]:
def divides(x, y):
    assert x != 0, "Bad argument to divides - denominator should be non-zero"
    if (type(x) != int or type(y) != int):
        raise TypeError("divides only takes integers")
    return y%x == 0

In [49]:
divides("cat",9)

TypeError: divides only takes integers

In [50]:
safe_apply_fun(divides24, "cat")

'Bad function or arg type'

## Exceptions are classes

They have constructors, selectors, methods, etc.

In [51]:
# Exceptions are classes too
class NoiseyException(Exception):
    def __init__(self, stuff):
        print("Bad stuff happenned", stuff)

In [52]:
def nostop(fun, x):
    try: 
        try:
            return fun(x)
        except:
            raise NoiseyException((fun, x))
    except:
        return None

In [53]:
def reciprocal(x):
    return 1/x
nostop(reciprocal, 0)

Bad stuff happenned (<function reciprocal at 0x1077f9158>, 0)


In [54]:
def zapper(fun, seq, selectors):
    return [nostop(fun, seq[x]) for x in selectors]

In [55]:
zapper(reciprocal, [1, 0, 2, 0], [0, 1, 2, 3])

Bad stuff happenned (<function reciprocal at 0x1077f9158>, 0)
Bad stuff happenned (<function reciprocal at 0x1077f9158>, 0)


[1.0, None, 0.5, None]

In [56]:
zapper(reciprocal, [1, 0, 2, 0], [0, 1, 2, 3, 4])

Bad stuff happenned (<function reciprocal at 0x1077f9158>, 0)
Bad stuff happenned (<function reciprocal at 0x1077f9158>, 0)


IndexError: list index out of range

In [57]:
def zing(seq, i):
    try: 
        try:
            return seq[i]
        except:
            raise NoiseyException(("bad sequence index", i))
    except:
        return None

def zapper(fun, seq, selectors):
    return [nostop(fun, zing(seq, x)) for x in selectors]

In [58]:
zapper(reciprocal, [1, 0, 2, 0], [0, 1, 2, 3, 4])

Bad stuff happenned (<function reciprocal at 0x1077f9158>, 0)
Bad stuff happenned (<function reciprocal at 0x1077f9158>, 0)
Bad stuff happenned ('bad sequence index', 4)
Bad stuff happenned (<function reciprocal at 0x1077f9158>, None)


[1.0, None, 0.5, None, None]

In [59]:
class NoiseyException(Exception):
    exceptions = []
    def __init__(self, stuff):
        print("Bad stuff happenned", stuff)
        NoiseyException.exceptions.append(stuff)

In [60]:
zapper(reciprocal, [1, 0, 2, 0], [0, 1, 2, 3, 4])

Bad stuff happenned (<function reciprocal at 0x1077f9158>, 0)
Bad stuff happenned (<function reciprocal at 0x1077f9158>, 0)
Bad stuff happenned ('bad sequence index', 4)
Bad stuff happenned (<function reciprocal at 0x1077f9158>, None)


[1.0, None, 0.5, None, None]

In [61]:
NoiseyException.exceptions

[(<function __main__.reciprocal>, 0),
 (<function __main__.reciprocal>, 0),
 ('bad sequence index', 4),
 (<function __main__.reciprocal>, None)]